In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, cohen_kappa_score
import matplotlib.pyplot as plt
import warnings
import os
import time
import joblib
import pickle
import random
warnings.filterwarnings('ignore')

print(f"{'='*80}")
print("全局模型 vs 分区模型性能比较（包含面积调整）")
print(f"{'='*80}")

# 设置随机种子保证可重复性
np.random.seed(42)
random.seed(42)

# 1. 加载数据
print("\n1. 加载数据...")
df = pd.read_csv('/content/drive/MyDrive/merged_data_by_year_生态区级 (2).csv')
df = df.dropna(subset=['ID'])
df['ID'] = df['ID'].astype(str).str.strip()

exclude_cols = ['lat', 'lon', 'year', 'ID', 'b1']
feature_cols = [col for col in df.columns if col not in exclude_cols]
target_col = 'b1'

# 计算全国的PFCL基础率（面积调整的基准）
national_base_rate = df['b1'].mean()
print(f"  全国PFCL基础率: {national_base_rate:.4%}")

print(f"  总数据量: {len(df)} 行")
print(f"  特征数量: {len(feature_cols)}")
print(f"  生态区数量: {df['ID'].nunique()}")

# 2. 为每个生态区划分训练集和测试集
print(f"\n2. 为每个生态区划分训练集(80%)和测试集(20%)...")

# 存储划分结果和本地模型
ecoregion_data = {}
global_train_samples = []

# 计算每个生态区的基础率（用于面积调整）
ecoregion_base_rates = {}

# 遍历每个生态区
for ecoregion_id in df['ID'].unique():
    eco_data = df[df['ID'] == ecoregion_id].copy()

    # 计算生态区基础率
    ecoregion_base_rate = eco_data['b1'].mean()
    ecoregion_base_rates[ecoregion_id] = ecoregion_base_rate

    if len(eco_data) < 10:  # 如果样本太少，跳过
        print(f"  警告: 生态区 {ecoregion_id} 样本数太少 ({len(eco_data)}), 跳过")
        continue

    # 划分训练集和测试集
    X = eco_data[feature_cols].values
    y = eco_data[target_col].values

    # 使用分层抽样，确保正负样本比例
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # 存储划分结果
    ecoregion_data[ecoregion_id] = {
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test,
        'n_total': len(eco_data),
        'positive_ratio': y.mean(),
        'base_rate': ecoregion_base_rate,  # 生态区基础率
        'local_model': None,
        'local_model_trained': False
    }

    # 收集全局训练样本
    for i in range(len(X_train)):
        global_train_samples.append({
            'features': X_train[i],
            'label': y_train[i],
            'ecoregion': ecoregion_id
        })

print(f"\n  全局训练样本总数: {len(global_train_samples)}")
print(f"  有效生态区数量: {len(ecoregion_data)}")

# 3. 从全局训练样本中随机抽取50%
print(f"\n3. 从全局训练样本中随机抽取50%用于训练全局模型...")
sample_size = int(len(global_train_samples) * 0.1)
sampled_indices = random.sample(range(len(global_train_samples)), sample_size)

# 构建全局训练集
X_global_train = np.array([global_train_samples[i]['features'] for i in sampled_indices])
y_global_train = np.array([global_train_samples[i]['label'] for i in sampled_indices])

print(f"  抽取的训练样本数: {len(X_global_train)}")
print(f"  正样本比例: {y_global_train.mean():.2%}")

# 4. 训练全局模型
print(f"\n4. 训练全局模型...")
rf_params = {
    'n_estimators': 100,
    'max_depth': 15,
    'min_samples_split': 20,
    'min_samples_leaf': 10,
    'max_features': 'sqrt',
    'class_weight': 'balanced',
    'random_state': 42,
    'n_jobs': -1,
    'verbose': 0
}

start_time = time.time()
global_model = RandomForestClassifier(**rf_params)
global_model.fit(X_global_train, y_global_train)
global_training_time = time.time() - start_time

print(f"  全局模型训练完成! 耗时: {global_training_time:.1f} 秒")

# 5. 训练每个生态区的本地模型
print(f"\n5. 训练每个生态区的本地模型...")
local_models = {}
local_training_times = []

for ecoregion_id, data in ecoregion_data.items():
    X_train = data['X_train']
    y_train = data['y_train']

    if len(X_train) < 20:
        print(f"  警告: 生态区 {ecoregion_id} 训练样本太少 ({len(X_train)}), 跳过本地模型训练")
        continue

    print(f"  训练生态区 {ecoregion_id} 的本地模型 ({len(X_train)} 个训练样本)...")

    start_time = time.time()
    local_model = RandomForestClassifier(**rf_params)
    local_model.fit(X_train, y_train)
    training_time = time.time() - start_time
    local_training_times.append(training_time)

    local_models[ecoregion_id] = local_model
    ecoregion_data[ecoregion_id]['local_model'] = local_model
    ecoregion_data[ecoregion_id]['local_model_trained'] = True

print(f"\n  本地模型训练完成!")
print(f"  共训练了 {len(local_models)} 个本地模型")

# 定义面积调整函数
def calculate_area_adjusted_metrics(y_true, y_pred_proba, base_rate, original_threshold=0.5):
    """
    计算面积调整后的指标

    参数:
    y_true: 真实标签
    y_pred_proba: 预测概率（正类的概率）
    base_rate: 实际基础率（PFCL在景观中的比例）
    original_threshold: 原始分类阈值（默认0.5，对应1:1平衡采样）

    返回:
    调整后的各项指标
    """
    # 根据基础率调整阈值
    # 贝叶斯调整：新阈值 = base_rate / (base_rate + (1-base_rate) * (1/原始阈值 - 1))
    adjusted_threshold = base_rate / (base_rate + (1-base_rate) * (1/original_threshold - 1))

    # 使用调整后的阈值进行分类
    y_pred_adjusted = (y_pred_proba >= adjusted_threshold).astype(int)

    # 计算混淆矩阵
    cm = confusion_matrix(y_true, y_pred_adjusted)
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
    else:
        # 如果只有一个类别
        if len(np.unique(y_pred_adjusted)) == 1:
            if y_pred_adjusted[0] == 0:
                tn, fp, fn, tp = len(y_true), 0, 0, 0
            else:
                tn, fp, fn, tp = 0, 0, 0, len(y_true)
        else:
            # 这种情况不应该发生，但以防万一
            return {k: np.nan for k in ['OA', 'F1', 'Kappa', 'PA', 'UA']}

    # 计算调整后的指标
    total = tn + fp + fn + tp

    if total > 0:
        oa_adj = (tn + tp) / total
    else:
        oa_adj = 0

    if tp + fn > 0:
        pa_adj = tp / (tp + fn)  # 生产者精度
    else:
        pa_adj = 0

    if tp + fp > 0:
        ua_adj = tp / (tp + fp)  # 用户精度
    else:
        ua_adj = 0

    if pa_adj + ua_adj > 0:
        f1_adj = 2 * (pa_adj * ua_adj) / (pa_adj + ua_adj)
    else:
        f1_adj = 0

    # 计算Kappa
    # 期望一致率 Pe
    p_observed = oa_adj
    p_expected = ((tn + fp) * (tn + fn) + (fn + tp) * (fp + tp)) / (total * total) if total > 0 else 0

    if 1 - p_expected > 0:
        kappa_adj = (p_observed - p_expected) / (1 - p_expected)
    else:
        kappa_adj = 0

    return {
        'OA_adj': oa_adj,
        'F1_adj': f1_adj,
        'Kappa_adj': kappa_adj,
        'PA_adj': pa_adj,
        'UA_adj': ua_adj,
        'adjusted_threshold': adjusted_threshold,
        'tn': tn,
        'fp': fp,
        'fn': fn,
        'tp': tp
    }

# 6. 在每个生态区的测试集上评估全局模型和本地模型（包含面积调整）
print(f"\n6. 在每个生态区的测试集上评估模型性能（包含面积调整）...")

results = []
national_test_data = []  # 收集全国测试数据，用于计算全国指标

for ecoregion_id, data in ecoregion_data.items():
    X_test = data['X_test']
    y_test = data['y_test']

    if len(X_test) == 0:
        continue

    # 收集全国测试数据
    for i in range(len(X_test)):
        national_test_data.append({
            'features': X_test[i],
            'label': y_test[i],
            'ecoregion': ecoregion_id
        })

    # 获取生态区基础率（用于面积调整）
    eco_base_rate = data['base_rate']

    # 使用全局模型预测
    y_pred_global = global_model.predict(X_test)
    y_pred_proba_global = global_model.predict_proba(X_test)[:, 1]

    # 计算全局模型原始指标（1:1平衡采样下的指标）
    oa_global = accuracy_score(y_test, y_pred_global)
    f1_global = f1_score(y_test, y_pred_global, zero_division=0)
    pa_global = recall_score(y_test, y_pred_global, zero_division=0)
    ua_global = precision_score(y_test, y_pred_global, zero_division=0)
    kappa_global = cohen_kappa_score(y_test, y_pred_global)

    # 计算全局模型面积调整后的指标（使用生态区基础率）
    global_adj_metrics = calculate_area_adjusted_metrics(
        y_test, y_pred_proba_global, eco_base_rate, original_threshold=0.5
    )

    # 初始化本地模型指标
    oa_local = np.nan
    f1_local = np.nan
    pa_local = np.nan
    ua_local = np.nan
    kappa_local = np.nan
    local_adj_metrics = {
        'OA_adj': np.nan,
        'F1_adj': np.nan,
        'Kappa_adj': np.nan,
        'PA_adj': np.nan,
        'UA_adj': np.nan
    }
    local_model_trained = data['local_model_trained']

    # 如果有本地模型，计算本地模型性能
    if local_model_trained and ecoregion_id in local_models:
        local_model = local_models[ecoregion_id]
        y_pred_local = local_model.predict(X_test)
        y_pred_proba_local = local_model.predict_proba(X_test)[:, 1]

        # 计算本地模型原始指标
        oa_local = accuracy_score(y_test, y_pred_local)
        f1_local = f1_score(y_test, y_pred_local, zero_division=0)
        pa_local = recall_score(y_test, y_pred_local, zero_division=0)
        ua_local = precision_score(y_test, y_pred_local, zero_division=0)
        kappa_local = cohen_kappa_score(y_test, y_pred_local)

        # 计算本地模型面积调整后的指标（使用生态区基础率）
        local_adj_metrics = calculate_area_adjusted_metrics(
            y_test, y_pred_proba_local, eco_base_rate, original_threshold=0.5
        )

    # 记录结果
    results.append({
        'ecoregion': ecoregion_id,
        'n_total': data['n_total'],
        'n_train': len(data['X_train']),
        'n_test': len(X_test),
        'positive_ratio': data['positive_ratio'],
        'base_rate': eco_base_rate,

        # 全局模型原始指标
        'OA_global': oa_global,
        'F1_global': f1_global,
        'Kappa_global': kappa_global,
        'PA_global': pa_global,
        'UA_global': ua_global,

        # 全局模型面积调整后指标
        'OA_global_adj': global_adj_metrics['OA_adj'],
        'F1_global_adj': global_adj_metrics['F1_adj'],
        'Kappa_global_adj': global_adj_metrics['Kappa_adj'],
        'PA_global_adj': global_adj_metrics['PA_adj'],
        'UA_global_adj': global_adj_metrics['UA_adj'],

        # 本地模型原始指标
        'OA_local': oa_local,
        'F1_local': f1_local,
        'Kappa_local': kappa_local,
        'PA_local': pa_local,
        'UA_local': ua_local,

        # 本地模型面积调整后指标
        'OA_local_adj': local_adj_metrics['OA_adj'],
        'F1_local_adj': local_adj_metrics['F1_adj'],
        'Kappa_local_adj': local_adj_metrics['Kappa_adj'],
        'PA_local_adj': local_adj_metrics['PA_adj'],
        'UA_local_adj': local_adj_metrics['UA_adj'],

        # 调整阈值
        'adjusted_threshold': global_adj_metrics['adjusted_threshold'],

        'local_model_trained': local_model_trained
    })

# 转换为DataFrame
results_df = pd.DataFrame(results)

# 7. 计算全国尺度的面积调整指标
print(f"\n7. 计算全国尺度的面积调整指标...")

if len(national_test_data) > 0:
    # 准备全国测试数据
    X_national_test = np.array([item['features'] for item in national_test_data])
    y_national_test = np.array([item['label'] for item in national_test_data])

    # 使用全局模型预测全国测试集
    y_pred_national = global_model.predict(X_national_test)
    y_pred_proba_national = global_model.predict_proba(X_national_test)[:, 1]

    # 计算全局模型原始指标（1:1平衡采样）
    oa_national = accuracy_score(y_national_test, y_pred_national)
    f1_national = f1_score(y_national_test, y_pred_national, zero_division=0)
    pa_national = recall_score(y_national_test, y_pred_national, zero_division=0)
    ua_national = precision_score(y_national_test, y_pred_national, zero_division=0)
    kappa_national = cohen_kappa_score(y_national_test, y_pred_national)

    # 计算全局模型面积调整后的指标（使用全国基础率）
    national_adj_metrics = calculate_area_adjusted_metrics(
        y_national_test, y_pred_proba_national, national_base_rate, original_threshold=0.5
    )

    print(f"  全国尺度全局模型性能:")
    print(f"    原始指标 - OA={oa_national:.4f}, F1={f1_national:.4f}, Kappa={kappa_national:.4f}, PA={pa_national:.4f}, UA={ua_national:.4f}")
    print(f"    面积调整后 - OA_adj={national_adj_metrics['OA_adj']:.4f}, F1_adj={national_adj_metrics['F1_adj']:.4f}, Kappa_adj={national_adj_metrics['Kappa_adj']:.4f}")
    print(f"                PA_adj={national_adj_metrics['PA_adj']:.4f}, UA_adj={national_adj_metrics['UA_adj']:.4f}")
    print(f"    调整阈值: {national_adj_metrics['adjusted_threshold']:.6f}")
else:
    print(f"  警告: 没有足够的全国测试数据")
    national_adj_metrics = {
        'OA_adj': np.nan,
        'F1_adj': np.nan,
        'Kappa_adj': np.nan,
        'PA_adj': np.nan,
        'UA_adj': np.nan
    }

# 8. 保存结果
print(f"\n8. 保存结果...")

# 创建汇总结果
summary_results = {
    'national_base_rate': national_base_rate,
    'OA_national_original': oa_national if len(national_test_data) > 0 else np.nan,
    'F1_national_original': f1_national if len(national_test_data) > 0 else np.nan,
    'Kappa_national_original': kappa_national if len(national_test_data) > 0 else np.nan,
    'PA_national_original': pa_national if len(national_test_data) > 0 else np.nan,
    'UA_national_original': ua_national if len(national_test_data) > 0 else np.nan,
    'OA_national_adj': national_adj_metrics['OA_adj'],
    'F1_national_adj': national_adj_metrics['F1_adj'],
    'Kappa_national_adj': national_adj_metrics['Kappa_adj'],
    'PA_national_adj': national_adj_metrics['PA_adj'],
    'UA_national_adj': national_adj_metrics['UA_adj'],
    'n_ecoregions': len(ecoregion_data),
    'n_test_samples_total': len(national_test_data) if len(national_test_data) > 0 else 0,
    'global_model_training_time': global_training_time
}

# 转换为DataFrame并保存
summary_df = pd.DataFrame([summary_results])
summary_path = '/content/drive/MyDrive/global_vs_local_area_adjusted_summary.csv'
summary_df.to_csv(summary_path, index=False)
print(f"  全国尺度汇总结果已保存到: {summary_path}")

# 保存详细结果
detailed_path = '/content/drive/MyDrive/global_vs_local_area_adjusted_detailed.csv'
results_df.to_csv(detailed_path, index=False)
print(f"  详细生态区结果已保存到: {detailed_path}")

# 9. 统计分析
print(f"\n9. 统计分析...")

# 筛选出有本地模型的生态区
valid_results = results_df[results_df['local_model_trained'] == True].copy()

if len(valid_results) > 0:
    print(f"  有本地模型的生态区数量: {len(valid_results)}")

    # 计算平均性能（面积调整后）
    mean_oa_global_adj = valid_results['OA_global_adj'].mean()
    mean_f1_global_adj = valid_results['F1_global_adj'].mean()
    mean_oa_local_adj = valid_results['OA_local_adj'].mean()
    mean_f1_local_adj = valid_results['F1_local_adj'].mean()

    print(f"\n  面积调整后的平均性能比较:")
    print(f"    全局模型 - 平均OA_adj: {mean_oa_global_adj:.3f}, 平均F1_adj: {mean_f1_global_adj:.3f}")
    print(f"    本地模型 - 平均OA_adj: {mean_oa_local_adj:.3f}, 平均F1_adj: {mean_f1_local_adj:.3f}")

    # 计算性能差异
    mean_f1_diff_adj = mean_f1_local_adj - mean_f1_global_adj
    mean_oa_diff_adj = mean_oa_local_adj - mean_oa_global_adj

    print(f"\n  面积调整后平均性能差异 (本地模型 - 全局模型):")
    print(f"    平均F1差异: {mean_f1_diff_adj:.3f}")
    print(f"    平均OA差异: {mean_oa_diff_adj:.3f}")

# 10. 生成最终报告
print(f"\n{'='*80}")
print("10. 总结报告")
print(f"{'='*80}")

report = f"""
全局模型 vs 分区模型性能比较报告（包含面积调整）
===========================================

实验设置:
- 全国PFCL基础率: {national_base_rate:.4%}
- 总生态区数量: {df['ID'].nunique()}
- 有效生态区数量: {len(ecoregion_data)}
- 全局训练样本数: {len(X_global_train)}
- 全国测试样本总数: {len(national_test_data)}

模型配置:
- 模型类型: 随机森林
- 树数量: {rf_params['n_estimators']}
- 最大深度: {rf_params['max_depth']}

全国尺度面积调整结果:
- 全国基础率: {national_base_rate:.4%}
- OA_national_original: {oa_national:.4f} -> OA_national_adj: {national_adj_metrics['OA_adj']:.4f}
- F1_national_original: {f1_national:.4f} -> F1_national_adj: {national_adj_metrics['F1_adj']:.4f}
- Kappa_national_original: {kappa_national:.4f} -> Kappa_national_adj: {national_adj_metrics['Kappa_adj']:.4f}
- PA_national_original: {pa_national:.4f} -> PA_national_adj: {national_adj_metrics['PA_adj']:.4f}
- UA_national_original: {ua_national:.4f} -> UA_national_adj: {national_adj_metrics['UA_adj']:.4f}
- 调整阈值: {national_adj_metrics['adjusted_threshold']:.6f}

关键发现:
"""

if len(valid_results) > 0:
    report += f"""
生态区尺度平均性能（面积调整后）:
- 全局模型平均F1_adj: {mean_f1_global_adj:.3f}
- 本地模型平均F1_adj: {mean_f1_local_adj:.3f}
- 平均F1差异 (本地-全局): {mean_f1_diff_adj:.3f}
"""

# 判断哪种模型更好
if 'mean_f1_diff_adj' in locals() and mean_f1_diff_adj > 0.05:
    report += "\n结论: 面积调整后，本地模型仍明显优于全局模型，生态区特异性建模具有优势。"
elif 'mean_f1_diff_adj' in locals() and mean_f1_diff_adj > 0:
    report += "\n结论: 面积调整后，本地模型略有优势，但差异不大。"
elif 'mean_f1_diff_adj' in locals() and mean_f1_diff_adj < 0:
    report += "\n结论: 面积调整后，全局模型反而表现更好，可能由于本地模型过拟合。"
else:
    report += "\n结论: 面积调整后，两种模型性能相当。"

report += f"""

输出文件:
1. {summary_path} - 全国尺度汇总结果（包含面积调整）
2. {detailed_path} - 详细生态区结果
3. /content/drive/MyDrive/global_model_comparison.pkl - 全局模型文件

面积调整的影响:
- UA（用户精度）通常会大幅下降，因为基础率降低意味着假阳性相对增加
- PA（生产者精度）变化较小，因为漏报率相对稳定
- OA（总体精度）通常会上升，因为负类样本远多于正类
- 调整阈值从0.5降低到{national_adj_metrics.get('adjusted_threshold', 0):.4f}，使模型对正类更敏感
"""

print(report)

# 保存报告
report_path = '/content/drive/MyDrive/global_vs_local_area_adjusted_report.txt'
with open(report_path, 'w') as f:
    f.write(report)

print(f"\n报告已保存到: {report_path}")
print(f"\n{'='*80}")
print("处理完成!")
print(f"{'='*80}")